In [1]:
# CMI BFRB Detection - LightGBM with World Acceleration Feature
# A simple LightGBM implementation for BFRB classification with world coordinate transformation
# 
# Key Innovation: Converting device acceleration to world coordinates using quaternion rotations
# This helps normalize hand orientation differences across subjects and positions
# Thanks https://www.kaggle.com/competitions/cmi-detect-behavior-with-sensor-data/discussion/583080 @tatamikenn for your idea!

import os
import optuna
import numpy as np
import pandas as pd
import polars as pl
import joblib
import cuda
from typing import Tuple, List, Optional
import warnings
warnings.filterwarnings('ignore')

# ML utilities
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedGroupKFold, PredefinedSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from lightgbm import LGBMClassifier, log_evaluation, early_stopping
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import clone
import xgboost as xgb

# World coordinate transformation
from scipy.spatial.transform import Rotation as R

# Competition specific
import kaggle_evaluation.cmi_inference_server

print("✓ All imports loaded successfully")

# =============================================================================
# CONFIGURATION
# =============================================================================

class Config: 
    """Central configuration class for training and data parameters"""
    IDS_DEL_PATH = "/kaggle/input/processdata/ids2del_prop_0.2.csv"
    TRAIN_PATH = "/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv"
    TRAIN_DEMOGRAPHICS_PATH = "/kaggle/input/cmi-detect-behavior-with-sensor-data/train_demographics.csv"
    TEST_PATH = "/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv"
    TEST_DEMOGRAPHICS_PATH = "/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv"
    
    # Training parameters
    SEED = 42
    N_FOLDS = 5
    
    # Feature columns
    ACC_COLS = ['acc_x', 'acc_y', 'acc_z']
    ROT_COLS = ['rot_w', 'rot_x', 'rot_y', 'rot_z']
    
    # LightGBM parameters
   
    XGB_PARAMS = { # Standard
        'objective': 'multi:softprob',
        'eval_metric': ['mlogloss', 'merror'],
        'n_estimators': 2000,
        'learning_rate': 0.05,
        'max_depth': 7,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'random_state': SEED,
        'n_jobs': -1
    }
    
    XGB_PARAMS2 = { # BaseLine
        'objective': 'multi:softmax',
        'eval_metric': ['mlogloss', 'merror'],
        'n_estimators': 2000,
        'learning_rate': 0.04707405163894676, 
        'max_depth': 7, 
        'min_child_weight': 2, 
        'subsample': 0.6185954480467879, 
        'colsample_bytree': 0.7754228640031244, 
        'reg_lambda': 0.4360516913148179, 
        'reg_alpha': 0.11248409929686172,
        'random_state': SEED,
        'n_jobs': -1
    }
    XGB_PARAMS3 = { # 0. 7458 ( 32 )
        'objective': 'multi:softmax', 
        'eval_metric': ['mlogloss', 'merror'], 
        'n_estimators': 5000, 
        'learning_rate': 0.02740219805275112, 
        'max_depth': 7, 
        'min_child_weight': 6, 
        'subsample': 0.7071091260648908, 
        'colsample_bytree': 0.7129247638273948, 
        'reg_lambda': 0.22204140017096774, 
        'reg_alpha': 0.19170683500427393, 
        'random_state': SEED, 
        'n_jobs': -1
    }
    XGB_PARAMS4 = { # 0.7449 ( 30 )
        'objective': 'multi:softmax', 
        'eval_metric': ['mlogloss', 'merror'], 
        'n_estimators': 5000, 
        'learning_rate': 0.04555686424007565,
        'max_depth': 7, 'min_child_weight': 6, 
        'subsample': 0.5955362936258931,
        'colsample_bytree': 0.7265766868210657, 
        'reg_lambda': 0.2714987313241283, 
        'reg_alpha': 0.17748628932990054, 
        'random_state': 42,
        'n_jobs': -1
    }
    XGB_PARAMS5 = { # 0.7442 ( 21 )
        'objective': 'multi:softmax',
        'eval_metric': ['mlogloss', 'merror'], 
        'n_estimators': 5000, 
        'learning_rate': 0.07104264055578176, 
        'max_depth': 7, 'min_child_weight': 7, 
        'subsample': 0.6881120987165036, 
        'colsample_bytree': 0.8560523003279517, 
        'reg_lambda': 2.9133504767574196,
        'reg_alpha': 0.803004286910833, 
        'random_state': 42, 
        'n_jobs': -1
    }
    XGB_PARAMS6 = { # 0.7440 ( 4 )
        'objective': 'multi:softprob',
        'eval_metric': ['mlogloss', 'merror'], 
        'n_estimators': 5000, 
        'learning_rate': 0.11042308668845532, 
        'max_depth': 6, 'min_child_weight': 6,
        'subsample': 0.61594051683421, 
        'colsample_bytree': 0.5980154856477609,
        'reg_lambda': 0.04094249505004808, 
        'reg_alpha': 0.056342092372682534, 
        'random_state': 42, 
        'n_jobs': -1}

# Set reproducibility
np.random.seed(Config.SEED)

✓ All imports loaded successfully


In [2]:
def check_gpu_availability():
    """Check if GPU is available for LightGBM"""
    try:
        # Try to create a simple LightGBM model with GPU
        from lightgbm import LGBMClassifier
        import numpy as np
        
        # Create dummy data
        X_dummy = np.random.rand(100, 10)
        y_dummy = np.random.randint(0, 2, 100)
        
        # Try GPU
        model = LGBMClassifier(n_estimators=1, device='gpu', verbosity=-1)
        model.fit(X_dummy, y_dummy)
        print("✓ GPU available for LightGBM")
        return 'gpu'
    except:
        print("⚠️  GPU not available, using CPU for LightGBM")
        return 'cpu'

# Check GPU availability
DEVICE = check_gpu_availability()

print(f"✓ Configuration loaded for Kaggle environment (Device: {DEVICE})")

1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


✓ GPU available for LightGBM
✓ Configuration loaded for Kaggle environment (Device: gpu)


1 warning generated.


In [3]:
GESTURE_MAPPER = {
    "Above ear - pull hair": 0,
    "Cheek - pinch skin": 1,
    "Eyebrow - pull hair": 2,
    "Eyelash - pull hair": 3, 
    "Forehead - pull hairline": 4,
    "Forehead - scratch": 5,
    "Neck - pinch skin": 6, 
    "Neck - scratch": 7,
    
    "Drink from bottle/cup": 8,
    "Feel around in tray and pull out an object": 9,
    "Glasses on/off": 10,
    "Pinch knee/leg skin": 11, 
    "Pull air toward your face": 12,
    "Scratch knee/leg skin": 13,
    "Text on phone": 14,
    "Wave hello": 15,
    "Write name in air": 16,
    "Write name on leg": 17,
}

REVERSE_GESTURE_MAPPER = {v: k for k, v in GESTURE_MAPPER.items()}

In [4]:
def competition_metric(y_true, y_pred) -> tuple:
    """Calculate the competition metric (Binary F1 + Macro F1) / 2"""
    
    # Binary F1: BFRB vs non-BFRB
    binary_f1 = f1_score(
        np.where(y_true <= 7, 1, 0),
        np.where(y_pred <= 7, 1, 0),
        zero_division=0.0,
    )
    
    # Macro F1: specific gesture classification (only for BFRB gestures)
    macro_f1 = f1_score(
        np.where(y_true <= 7, y_true, 99),  # Map non-BFRB to 99
        np.where(y_pred <= 7, y_pred, 99),  # Map non-BFRB to 99
        average="macro", 
        zero_division=0.0,
    )
    
    # Final competition score
    final_score = 0.5 * (binary_f1 + macro_f1)
    
    return final_score, binary_f1, macro_f1

In [5]:
def handle_quaternion_missing_values(rot_data: np.ndarray) -> np.ndarray:
    """
    Handle missing values in quaternion data intelligently
    
    Key insight: Quaternions must have unit length |q| = 1
    If one component is missing, we can reconstruct it from the others
    """
    rot_cleaned = rot_data.copy()
    
    for i in range(len(rot_data)):
        row = rot_data[i]
        missing_count = np.isnan(row).sum()
        
        if missing_count == 0:
            # No missing values, normalize to unit quaternion
            norm = np.linalg.norm(row)
            if norm > 1e-8:
                rot_cleaned[i] = row / norm
            else:
                rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]  # Identity quaternion
                
        elif missing_count == 1:
            # One missing value, reconstruct using unit quaternion constraint
            # |w|² + |x|² + |y|² + |z|² = 1
            missing_idx = np.where(np.isnan(row))[0][0]
            valid_values = row[~np.isnan(row)]
            
            sum_squares = np.sum(valid_values**2)
            if sum_squares <= 1.0:
                missing_value = np.sqrt(max(0, 1.0 - sum_squares))
                # Choose sign for continuity with previous quaternion
                if i > 0 and not np.isnan(rot_cleaned[i-1, missing_idx]):
                    if rot_cleaned[i-1, missing_idx] < 0:
                        missing_value = -missing_value
                rot_cleaned[i, missing_idx] = missing_value
                rot_cleaned[i, ~np.isnan(row)] = valid_values
            else:
                rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]
        else:
            # More than one missing value, use identity quaternion
            rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]
    
    return rot_cleaned

In [6]:
#Compute World Accc
def compute_world_acceleration(acc: np.ndarray, rot: np.ndarray) -> np.ndarray:
    """
    Convert acceleration from device coordinates to world coordinates
    
    This is the key innovation: normalizing for device orientation
    
    Args:
        acc: acceleration in device coordinates, shape (time_steps, 3) [x, y, z]
        rot: rotation quaternion, shape (time_steps, 4) [w, x, y, z] (normalized)
    
    Returns:
        acc_world: acceleration in world coordinates, shape (time_steps, 3)
        
    Why this matters:
    - Device acceleration depends on how the watch is oriented on the wri/st
    - World acceleration is independent of device orientation
    - This helps the model focus on actual hand motion rather than wrist rotation
    """
    try:
        # Convert quaternion format from [w, x, y, z] to [x, y, z, w] for scipy
        rot_scipy = rot[:, [1, 2, 3, 0]]
        
        # Verify quaternions are valid (non-zero norm)
        norms = np.linalg.norm(rot_scipy, axis=1)
        if np.any(norms < 1e-8):
            # Replace problematic quaternions with identity
            mask = norms < 1e-8
            rot_scipy[mask] = [0.0, 0.0, 0.0, 1.0]  # Identity quaternion in scipy format
        
        # Create rotation object and apply transformation
        r = R.from_quat(rot_scipy)
        acc_world = r.apply(acc)
        
    except Exception:
        # Fallback to original acceleration if transformation fails
        print("Warning: World coordinate transformation failed, using device coordinates")
        acc_world = acc.copy()
    
    return acc_world

#bỏ gia tốc trọng trường
def remove_gravity_from_acc(acc_data, rot_data):

    if isinstance(acc_data, pd.DataFrame):
        acc_values = acc_data[['acc_x', 'acc_y', 'acc_z']].values
    else:
        acc_values = acc_data

    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = acc_values.shape[0]
    linear_accel = np.zeros_like(acc_values)
    
    gravity_world = np.array([0, 0, 9.81])

    for i in range(num_samples):
        if np.all(np.isnan(quat_values[i])) or np.all(np.isclose(quat_values[i], 0)):
            linear_accel[i, :] = acc_values[i, :] 
            continue

        try:
            rotation = R.from_quat(quat_values[i])
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i, :] = acc_values[i, :] - gravity_sensor_frame
        except ValueError:
             linear_accel[i, :] = acc_values[i, :]
             
    return linear_accel

In [7]:
def calculate_angular_distance(rot_data: np.ndarray) -> np.ndarray:
    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_dist = np.zeros(num_samples)

    for i in range(num_samples - 1):
        q1 = quat_values[i]
        q2 = quat_values[i+1]

        if np.all(np.isnan(q1)) or np.all(np.isclose(q1, 0)) or \
           np.all(np.isnan(q2)) or np.all(np.isclose(q2, 0)):
            angular_dist[i] = 0 # Или np.nan, в зависимости от желаемого поведения
            continue
        try:
            # Преобразование кватернионов в объекты Rotation
            r1 = R.from_quat(q1)
            r2 = R.from_quat(q2)

            # Вычисление углового расстояния: 2 * arccos(|real(p * q*)|)
            # где p* - сопряженный кватернион q
            # В scipy.spatial.transform.Rotation, r1.inv() * r2 дает относительное вращение.
            # Угол этого относительного вращения - это и есть угловое расстояние.
            relative_rotation = r1.inv() * r2
            
            # Угол rotation vector соответствует угловому расстоянию
            # Норма rotation vector - это угол в радианах
            angle = np.linalg.norm(relative_rotation.as_rotvec())
            angular_dist[i] = angle
        except ValueError:
            angular_dist[i] = 0 # В случае недействительных кватернионов
            pass
            
    return angular_dist

In [8]:
#tính vận tốc quay
def calculate_angular_velocity_from_quat(rot_data: np.ndarray, time_delta=1/200) -> np.ndarray: # Assuming 200Hz sampling rate
    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_vel = np.zeros((num_samples, 3))

    for i in range(num_samples - 1):
        q_t = quat_values[i]
        q_t_plus_dt = quat_values[i+1]

        if np.all(np.isnan(q_t)) or np.all(np.isclose(q_t, 0)) or \
           np.all(np.isnan(q_t_plus_dt)) or np.all(np.isclose(q_t_plus_dt, 0)):
            continue

        try:
            rot_t = R.from_quat(q_t)
            rot_t_plus_dt = R.from_quat(q_t_plus_dt)

            # Calculate the relative rotation
            delta_rot = rot_t.inv() * rot_t_plus_dt
            
            # Convert delta rotation to angular velocity vector
            # The rotation vector (Euler axis * angle) scaled by 1/dt
            # is a good approximation for small delta_rot
            angular_vel[i, :] = delta_rot.as_rotvec() / time_delta
        except ValueError:
            # If quaternion is invalid, angular velocity remains zero
            pass
            
    return angular_vel

In [9]:
def mirror_quaternion(quat):
    """
    Mirror a single quaternion through the YZ plane.

    Args:
        quat (array of shape (N, 4)): [w, x, y, z]

    Returns:
        mirrored (np.ndarray of shape (N, 4)): mirrored quaternion [w, x, y, z]
    """

    P = np.diag([-1, 1, 1])  # reflection through YZ
    rot = R.from_quat(quat[:, [1, 2, 3, 0]])  # SciPy uses [x, y, z, w]
    R_mat = rot.as_matrix()
    R_flipped = P @ R_mat @ P
    flipped = R.from_matrix(R_flipped).as_quat()
    return flipped[:, [3, 0, 1, 2]]  # back to [w, x, y, z]


def mirror_data(data):
    """
    Mirror left-handed samples to match right-handed frame.

    Args:
        data (np.ndarray of shape (N, 7)): sensor data
    
    Returns:
        A new array with mirrored left-handed samples.

    """
    
    data[:, 0] = -data[:, 0]
    data[:, 3:] = mirror_quaternion(data[:, 3:]) # [w, x, y, z]

    return data

def process_left_handed(df, dem):
    
    left_handed = dem[dem["handedness"] == 0]
    left_handed = df.loc[df["subject"].isin(left_handed["subject"])]
    cols_to_transform = ["acc_x", "acc_y", "acc_z", "rot_w", "rot_x", "rot_y", "rot_z"]
    left_handed_arr = left_handed[cols_to_transform].to_numpy()
    df.loc[df["subject"].isin(left_handed["subject"]), cols_to_transform] = mirror_data(left_handed_arr)
    return df

In [10]:
def extract_comprehensive_features(sequence: pl.DataFrame, demographics: pl.DataFrame) -> pd.DataFrame:
    """
    Extract features from IMU data with world acceleration transformation
    
    Feature Groups:
    1. Device Acceleration (acc_x, acc_y, acc_z) - raw sensor data
    2. Rotation Quaternion (rot_w, rot_x, rot_y, rot_z) - device orientation  
    3. World Acceleration (NEW) - orientation-normalized acceleration
    4. Demographics - subject characteristics
    5. Sequence metadata - length, etc.
    """
    seq_df = sequence.to_pandas()
    demo_df = demographics.to_pandas()
    
    # Convert to pandas for processing
    seq_df = sequence.to_pandas()
    demo_df = demographics.to_pandas()
    
    rot_data = seq_df[Config.ROT_COLS].copy()
    rot_data = rot_data.ffill().bfill()
    rot_data_clean = handle_quaternion_missing_values(rot_data.values)
    
    seq_df[Config.ROT_COLS] = rot_data_clean
    
    if len(demo_df) > 0 and demo_df.iloc[0].get("handedness", 1) == 0:
        seq_df = process_left_handed(seq_df, demo_df)
   

     # Handle missing values in basic sensor data
    acc_data = seq_df[Config.ACC_COLS].copy()
    acc_data = acc_data.ffill().bfill().fillna(0)

    rot_data = seq_df[Config.ROT_COLS].copy()
    rot_data = rot_data.ffill().bfill()
    rot_data_clean = handle_quaternion_missing_values(rot_data.values)

    # rot_data_clean = seq_df[Config.ROT_COLS].copy()

    #linear acc
    try:
        acc_gravity_removed = remove_gravity_from_acc(acc_data.values, rot_data_clean)
        # print("✓ Gravity remove successfully")  # Reduced verbosity
    except Exception as e:
        print(f"Warning: Gravity remove failed: {e}")
        acc_gravity_removed = acc_data.values.copy()  # Fallback to device coordinates

    #linear world acc
    try:
        world_acc_data = compute_world_acceleration(acc_gravity_removed, rot_data_clean)
        # print("✓ World acceleration computed successfully")  # Reduced verbosity
    except Exception as e:
        print(f"Warning: World acceleration computation failed: {e}")
        world_acc_data = acc_gravity_removed.values.copy()  # Fallback to device coordinates  

    #angular velocity
    try:
        angular_velocity = calculate_angular_velocity_from_quat(rot_data_clean, time_delta=1/200)
        # print("✓ Calculate angular velocity successfully")  # Reduced verbosity
    except Exception as e:
        print(f"Warning: Calculate angular velocity failed: {e}")
        angular_velocity = rot_data.values.copy()  # Fallback to device coordinates

    #angular distance
    try:
        angular_distance = calculate_angular_distance(rot_data_clean)
        # print("✓ Calculate angular distance successfully")  # Reduced verbosity
    except Exception as e:
        print(f"Warning: Calculate angular distance failed: {e}")
        angular_distance = rot_data.values.copy()  # Fallback to device coordinates
    
    # Initialize feature dictionary
    features = {}
    
    # Add sequence metadata
    features['sequence_length'] = len(seq_df)
    
    # Add demographics features
    if len(demo_df) > 0:
        demo_row = demo_df.iloc[0]
        features['age'] = demo_row.get('age', 0)
        features['adult_child'] = demo_row.get('adult_child', 0)
        features['sex'] = demo_row.get('sex', 0)
        features['handedness'] = demo_row.get('handedness', 0)
        features['height_cm'] = demo_row.get('height_cm', 0)
        features['shoulder_to_wrist_cm'] = demo_row.get('shoulder_to_wrist_cm', 0)
        features['elbow_to_wrist_cm'] = demo_row.get('elbow_to_wrist_cm', 0)
    
    # Define feature arrays for statistical extraction
    feature_arrays = {
        'acc': acc_data.values,           # Device acceleration (3D)
        'rot': rot_data_clean,            # Rotation quaternion (4D) 
        'world_acc': world_acc_data,      # World acceleration (3D) - KEY INNOVATION
        'acc_g_remove': acc_gravity_removed,
        'ang_vel': angular_velocity,
        'ang_dis': angular_distance,
    }
    
    # Extract statistical features for each data source
    for source_name, array in feature_arrays.items():
        if array.ndim == 1:
            array = array.reshape(-1, 1)
        
        n_features = array.shape[1]
        
        for feat_idx in range(n_features):
            feat_data = array[:, feat_idx]
            
            # Create feature name
            if source_name == 'acc':
                axis_names = ['x', 'y', 'z']
                prefix = f"acc_{axis_names[feat_idx]}"
            elif source_name == 'rot':
                comp_names = ['w', 'x', 'y', 'z']
                prefix = f"rot_{comp_names[feat_idx]}"
            elif source_name == 'world_acc':
                axis_names = ['x', 'y', 'z']  
                prefix = f"world_acc_{axis_names[feat_idx]}"
            else:
                prefix = f"{source_name}_{feat_idx}" if n_features > 1 else source_name
            
            # Extract comprehensive statistical features
            features.update(extract_statistical_features(feat_data, prefix))
    
    # Compute magnitude features (important for motion intensity)
    acc_magnitude = np.linalg.norm(acc_data.values, axis=1)
    features.update(extract_statistical_features(acc_magnitude, 'acc_magnitude_raw'))

    # Compute linear_acc_magnitude
    linear_acc_magnitude = np.linalg.norm(acc_gravity_removed, axis=1)
    features.update(extract_statistical_features(linear_acc_magnitude, 'linear_acc_magnitude'))

    world_acc_magnitude = np.linalg.norm(world_acc_data, axis=1)
    features.update(extract_statistical_features(world_acc_magnitude, 'world_acc_magnitude'))
    
    # Cross-feature: difference between device and world acceleration magnitudes
    # This captures how much device orientation affects motion measurement
    acc_world_diff = linear_acc_magnitude - world_acc_magnitude
    features.update(extract_statistical_features(acc_world_diff, 'acc_world_diff'))
    
    # Convert to DataFrame
    result_df = pd.DataFrame([features])
    
    # Handle any remaining NaN values
    result_df = result_df.fillna(0)
    
    return result_df

In [11]:
def extract_statistical_features(data: np.ndarray, prefix: str) -> dict:
    """
    Extract comprehensive statistical features from a 1D time series
    
    Returns features that capture:
    - Central tendency: mean, median, mode region
    - Spread: std, variance, range, IQR  
    - Shape: skewness, kurtosis
    - Dynamics: differences, trends, changes
    - Segments: beginning vs middle vs end behavior
    """
    
    features = {}
    
    # Basic statistics
    features[f'{prefix}_mean'] = np.mean(data)
    features[f'{prefix}_std'] = np.std(data)
    features[f'{prefix}_var'] = np.var(data)
    features[f'{prefix}_min'] = np.min(data)
    features[f'{prefix}_max'] = np.max(data)
    features[f'{prefix}_median'] = np.median(data)
    features[f'{prefix}_q25'] = np.percentile(data, 25)
    features[f'{prefix}_q75'] = np.percentile(data, 75)
    features[f'{prefix}_iqr'] = np.percentile(data, 75) - np.percentile(data, 25)
    features[f'{prefix}_mean_abs'] = np.abs(data).mean()
    features[f'{prefix}_rms'] = np.sqrt((data**2).mean())
    
    # Range and boundary features
    features[f'{prefix}_range'] = np.max(data) - np.min(data)
    features[f'{prefix}_first'] = data[0] if len(data) > 0 else 0
    features[f'{prefix}_last'] = data[-1] if len(data) > 0 else 0
    features[f'{prefix}_delta'] = data[-1] - data[0] if len(data) > 0 else 0
    
    # Higher order moments (shape of distribution)
    if len(data) > 1 and np.std(data) > 1e-8:
        features[f'{prefix}_skew'] = pd.Series(data).skew()
        features[f'{prefix}_kurt'] = pd.Series(data).kurtosis()
    else:
        features[f'{prefix}_skew'] = 0
        features[f'{prefix}_kurt'] = 0
    
    # Differential features (capture dynamics)
    if len(data) > 1:
        diff_data = np.diff(data)
        features[f'{prefix}_diff_mean'] = np.mean(diff_data)
        features[f'{prefix}_diff_std'] = np.std(diff_data)
        features[f'{prefix}_n_changes'] = np.sum(np.abs(diff_data) > np.std(data) * 0.1)  # Significant changes
    else:
        features[f'{prefix}_diff_mean'] = 0
        features[f'{prefix}_diff_std'] = 0
        features[f'{prefix}_n_changes'] = 0
    
    # Correlation with time (trend detection)
    if len(data) > 2:
        time_indices = np.arange(len(data))
        try:
            corr_coef = np.corrcoef(time_indices, data)[0, 1]
            features[f'{prefix}_time_corr'] = corr_coef if not np.isnan(corr_coef) else 0
        except:
            features[f'{prefix}_time_corr'] = 0
    else:
        features[f'{prefix}_time_corr'] = 0
    
    # Segment features (beginning, middle, end patterns)
    seq_len = len(data)
    if seq_len >= 9:  # Need sufficient data for meaningful segments
        seg_size = seq_len // 3
        seg1 = data[:seg_size]           # Beginning (Transition phase)
        seg2 = data[seg_size:2*seg_size] # Middle (Pause phase)  
        seg3 = data[2*seg_size:]         # End (Gesture phase)
        
        features[f'{prefix}_seg1_mean'] = np.mean(seg1)
        features[f'{prefix}_seg2_mean'] = np.mean(seg2)
        features[f'{prefix}_seg3_mean'] = np.mean(seg3)
        
        features[f'{prefix}_seg1_std'] = np.std(seg1)
        features[f'{prefix}_seg2_std'] = np.std(seg2)
        features[f'{prefix}_seg3_std'] = np.std(seg3)
        
        # Segment transitions (important for distinguishing gesture types)
        features[f'{prefix}_seg1_to_seg2'] = np.mean(seg2) - np.mean(seg1)
        features[f'{prefix}_seg2_to_seg3'] = np.mean(seg3) - np.mean(seg2)
    else:
        # Not enough data for meaningful segments
        for seg in [1, 2, 3]:
            features[f'{prefix}_seg{seg}_mean'] = features[f'{prefix}_mean']
            features[f'{prefix}_seg{seg}_std'] = features[f'{prefix}_std']
        features[f'{prefix}_seg1_to_seg2'] = 0
        features[f'{prefix}_seg2_to_seg3'] = 0

    if len(data) > 1:
        series = pd.Series(data)

        # Peak detection - count significant changes
        features[f"{prefix}_peak_count"] = (series.diff().abs() > (series.std() * 0.5)).sum()

        # Movement intensity - mean absolute deviation (MAD)
        features[f"{prefix}_mad"] = (series - series.mean()).abs().mean()

        # Range normalization - signal range relative to std
        features[f"{prefix}_range_norm"] = (series.max() - series.min()) / (series.std() + 1e-8)

        # Acceleration analysis - second derivative
        accel = series.diff().diff()
        features[f"{prefix}_accel_mean"] = accel.abs().mean()
        features[f"{prefix}_accel_std"] = accel.std()

        # Movement consistency - coefficient of variation (CV)
        features[f"{prefix}_cv"] = series.std() / (abs(series.mean()) + 1e-8)

        # Energy measure - sum of squared values
        features[f"{prefix}_energy"] = (series ** 2).sum()
    else:
        features[f"{prefix}_peak_count"] = 0
        features[f"{prefix}_mad"] = 0
        features[f"{prefix}_range_norm"] = 0
        features[f"{prefix}_accel_mean"] = 0
        features[f"{prefix}_accel_std"] = 0
        features[f"{prefix}_cv"] = 0
        features[f"{prefix}_energy"] = 0
    
    return features

In [12]:
def load_and_prepare_data():
    """Load and prepare training data with comprehensive features"""
    
    print("Loading training data...")
    train_df = pl.read_csv(Config.TRAIN_PATH)
    train_demographics = pl.read_csv(Config.TRAIN_DEMOGRAPHICS_PATH)

    # tmp = train_df.group_by(['subject','sequence_id','phase']).agg(pl.len()
    #       .alias('count')).pivot(values='count', index=['subject','sequence_id'], columns='phase')
    # tmp = tmp.with_columns((pl.col('Gesture') + pl.col('Transition')).alias('sum'))
    # tmp = tmp.with_columns([
    # (pl.col('Gesture') / pl.col('sum') * 100).alias('Gesture_Ratio'),
    # (pl.col('Transition') / pl.col('sum') * 100).alias('Transition_Ratio'),
    # ])

    # gesture_ids = tmp.filter(pl.col('Gesture_Ratio') < 5).select('sequence_id').to_series().to_numpy()
    # transition_ids = tmp.filter(pl.col('Transition_Ratio') < 15).select('sequence_id').to_series().to_numpy()

    
    # remove_ids = np.unique(np.concatenate([gesture_ids, transition_ids]))

    # print(f"Number of gesture_ids flagged: {len(gesture_ids)}")
    # print(f"Unique remove_ids: {len(remove_ids)}")
    # print("Remove IDs:", remove_ids.tolist())

    
    # train_df = train_df.filter(~pl.col('sequence_id').is_in(remove_ids)).with_row_index().drop('index')

    # delete_id = [
    #     'SEQ_011975',
    # ]
    
    # print("Xóa Sequence")
    # delete_id = list(set(delete_id))  # loại bỏ trùng lặp
    # train_df = train_df.filter(~train_df["sequence_id"].is_in(delete_id))
    ids2del = pl.read_csv(Config.IDS_DEL_PATH)
    train_df = train_df.filter(~pl.col("sequence_id").is_in(ids2del["sequence_id"]))

    print(f"✅ Filtered train_df shape: {train_df.shape}")
    
    print("Loading test data...")
    test_df = pl.read_csv(Config.TEST_PATH)
    test_demographics = pl.read_csv(Config.TEST_DEMOGRAPHICS_PATH)
    
    # Get common columns between train and test (exclude thermal and ToF sensors)
    train_cols = set(train_df.columns)
    test_cols = set(test_df.columns)
    common_cols = train_cols.intersection(test_cols)
    
    # Filter to IMU-only columns (remove thermal and ToF sensors)
    imu_cols = [col for col in common_cols if not (col.startswith('thm_') or col.startswith('tof_'))]
    
    print(f"✓ Using {len(imu_cols)} common IMU columns : {imu_cols}")
    print(f"✓ Train-only columns: {train_cols - test_cols}")
    print(f"✓ Test-only columns: {test_cols - train_cols}")
    
    print("Extracting features for training sequences...")
    train_features_list = []
    train_labels = []
    train_subjects = []
    train_sequence_ids = []
    
    # Group by sequence_id for training data - need to include gesture column for labels
    train_imu_cols = imu_cols + ['gesture'] if 'gesture' not in imu_cols else imu_cols
    train_sequences = train_df.select(pl.col(train_imu_cols)).group_by('sequence_id', maintain_order=True)
    
    for sequence_id, sequence_data in train_sequences:
        # Get sequence features
        sequence_id_val = sequence_id[0] if isinstance(sequence_id, tuple) else sequence_id
        
        # Get demographics for this sequence
        subject_id = sequence_data['subject'][0]
        subject_demographics = train_demographics.filter(pl.col('subject') == subject_id)
        
        # Extract features (only IMU columns for feature extraction)
        imu_only_data = sequence_data.select(pl.col(imu_cols))
        features = extract_comprehensive_features(imu_only_data, subject_demographics)
        features['sequence_id'] = sequence_id_val
        
        train_features_list.append(features)
        
        # Get label (gesture) for this sequence
        gesture = sequence_data['gesture'][0]
        label = GESTURE_MAPPER[gesture]
        train_labels.append(label)
        train_subjects.append(subject_id)
        train_sequence_ids.append(sequence_id_val)
    
    # Combine all training features
    X_train = pd.concat(train_features_list, ignore_index=True)
    y_train = np.array(train_labels)
    subjects = np.array(train_subjects)
    
    print("Extracting features for test sequences...")
    test_features_list = []
    test_sequence_ids = []
    
    # Group by sequence_id for test data  
    test_sequences = test_df.select(pl.col(imu_cols)).group_by('sequence_id', maintain_order=True)
    
    for sequence_id, sequence_data in test_sequences:
        sequence_id_val = sequence_id[0] if isinstance(sequence_id, tuple) else sequence_id
        
        # Get demographics for this sequence
        subject_id = sequence_data['subject'][0]
        subject_demographics = test_demographics.filter(pl.col('subject') == subject_id)
        
        # Extract features using the same function as training
        features = extract_comprehensive_features(sequence_data, subject_demographics)
        features['sequence_id'] = sequence_id_val
        
        test_features_list.append(features)
        test_sequence_ids.append(sequence_id_val)
    
    # Combine all test features
    X_test = pd.concat(test_features_list, ignore_index=True)
    
    print(f"✓ Training features shape: {X_train.shape}")
    print(f"✓ Training labels shape: {y_train.shape}")
    print(f"✓ Test features shape: {X_test.shape}")
    print(f"✓ Number of features: {X_train.shape[1] - 1}")  # -1 for sequence_id
    
    return X_train, y_train, subjects, X_test, test_sequence_ids, imu_cols


In [13]:
def train_xgboost_models(X_train, y_train, subjects):
    """Train XGBoost models using stratified group k-fold cross-validation"""
    
    print(f"Training XGBoost models with {Config.N_FOLDS}-fold cross-validation...")
    if 0:
        best_params = tune_xgboost(X_train, y_train, subjects, n_trials=40)
    else:
        best_params = Config.XGB_PARAMS6
    # Prepare features (remove sequence_id)
    feature_cols = [col for col in X_train.columns if col != 'sequence_id']
    X_features = X_train[feature_cols]
    
    # Setup cross-validation
    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
    
    models = []
    oof_predictions = np.zeros(len(y_train), dtype=int)
    cv_scores = []
    
    print(f"Feature columns: {len(feature_cols)}")
    print("Starting cross-validation...")
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X_features, y_train, subjects)):
        print(f"\n--- Fold {fold + 1}/{Config.N_FOLDS} ---")
        
        # Split data
        X_fold_train = X_features.iloc[train_idx]
        X_fold_val = X_features.iloc[val_idx]
        y_fold_train = y_train[train_idx]
        y_fold_val = y_train[val_idx]
        
        print(f"Train size: {len(X_fold_train)}, Val size: {len(X_fold_val)}")
        
        # Adjust tree_method for GPU if available and desired
        params = best_params.copy()
        if DEVICE == 'gpu':
            params['tree_method'] = 'hist'
            params['device'] = 'cuda'
        
        model = xgb.XGBClassifier(**params)
        
        print(f"Training fold {fold + 1} with early stopping (Device: {DEVICE})...")
        model.fit(
            X_fold_train, y_fold_train,
            eval_set=[(X_fold_val, y_fold_val)],
            early_stopping_rounds=100,
            verbose=100
        )
        
        # Predictions
        val_preds = model.predict(X_fold_val)
        oof_predictions[val_idx] = val_preds
        
        # Calculate metrics
        score, binary_f1, macro_f1 = competition_metric(y_fold_val, val_preds)
        cv_scores.append(score)
        
        print(f"Fold {fold + 1} - Competition Score: {score:.4f} (Binary F1: {binary_f1:.4f}, Macro F1: {macro_f1:.4f})")
        
        models.append(model)
    
    # Overall CV performance
    overall_score, overall_binary_f1, overall_macro_f1 = competition_metric(y_train, oof_predictions)
    
    print(f"\n{'='*60}")
    print("CROSS-VALIDATION RESULTS")
    print(f"{'='*60}")
    print(f"Overall Competition Score: {overall_score:.4f} ± {np.std(cv_scores):.4f}")
    print(f"Overall Binary F1: {overall_binary_f1:.4f}")
    print(f"Overall Macro F1: {overall_macro_f1:.4f}")
    print(f"Fold scores: {[f'{score:.4f}' for score in cv_scores]}")
    print(f"{'='*60}")
    
    return models, cv_scores, overall_score

In [14]:
def tune_xgboost(X_train, y_train, subjects, n_trials=40):
    """Hyperparameter tuning for XGBoost using Optuna"""
    def objective(trial):
        # Suggest hyperparameters
        params = {
        'objective': 'multi:softprob',
        'num_class': len(np.unique(y_train)),
        'eval_metric': ['mlogloss', 'merror'],
        'n_estimators': 5000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 5, 8),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'random_state': Config.SEED,
        'n_jobs': -1,
        }
        
        print(f"\n[Optuna Trial {trial.number}] Testing params: {params}")
        
        cv = StratifiedGroupKFold(n_splits=Config.N_FOLDS, shuffle=True, random_state=Config.SEED)
        oof_predictions = np.zeros(len(y_train), dtype=int)

        feature_cols = [col for col in X_train.columns if col != 'sequence_id']
        X_features = X_train[feature_cols]
        
        
        for train_idx, val_idx in cv.split(X_features, y_train, subjects):
            X_tr, X_val = X_features.iloc[train_idx], X_features.iloc[val_idx]
            y_tr, y_val = y_train[train_idx], y_train[val_idx]

            params_copy = params.copy()
            if DEVICE == 'gpu':
                params_copy['tree_method'] = 'hist'
                params_copy['device'] = 'cuda'
            model = xgb.XGBClassifier(**params_copy)

            model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=100,
            verbose=100
            )
        
            val_preds = model.predict(X_val)
            oof_predictions[val_idx] = val_preds
            score, binary_f1, macro_f1 = competition_metric(y_val, val_preds)
            print(f"[Optuna Trial {trial.number}] Competition Score: {score:.4f} (Binary F1: {binary_f1:.4f}, Macro F1: {macro_f1:.4f}")

        overall_score, overall_binary_f1, overall_macro_f1 = competition_metric(y_train, oof_predictions)
        print(f"Competition Score: {overall_score:.4f} (Binary F1: {overall_binary_f1:.4f}, Macro F1: {overall_macro_f1:.4f}")
        return overall_score
        
        
        # Run Optuna study
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)
          
    print("Best trial:", study.best_trial.params)
    final_params = Config.XGB_PARAMS.copy()
    final_params.update(study.best_trial.params)
    return final_params

In [15]:
"""Main execution pipeline"""
print("="*60)
print("CMI BFRB Detection ")
# lstm_embeddings = joblib.load("/kaggle/input/lstm-embeddings/df_oof_embeddings.pkl")

# Load and prepare data
X_train, y_train, subjects, X_test, test_sequence_ids, imu_cols = load_and_prepare_data()



CMI BFRB Detection 
Loading training data...
✅ Filtered train_df shape: (571386, 341)
Loading test data...
✓ Using 11 common IMU columns : ['subject', 'rot_w', 'row_id', 'acc_z', 'acc_x', 'rot_z', 'sequence_counter', 'sequence_id', 'rot_x', 'acc_y', 'rot_y']
✓ Train-only columns: {'orientation', 'behavior', 'gesture', 'sequence_type', 'phase'}
✓ Test-only columns: set()
Extracting features for training sequences...
Extracting features for test sequences...
✓ Training features shape: (8109, 765)
✓ Training labels shape: (8109,)
✓ Test features shape: (2, 765)
✓ Number of features: 764


In [16]:

print("Merging...")
# X_train_merged = X_train.merge(lstm_embeddings, on="sequence_id", how="inner")
print("Merging Succesful")
models, cv_scores, overall_score = train_xgboost_models(X_train, y_train, subjects)
# Prepare feature columns for inference
feature_cols = [col for col in X_train.columns if col != 'sequence_id']
    
   
print(f"\n✓ Training completed successfully!")
print(f"✓ Final CV Score: {overall_score:.4f}")
print(f"✓ Models ready for inference")

Merging...
Merging Succesful
Training XGBoost models with 5-fold cross-validation...
Feature columns: 764
Starting cross-validation...

--- Fold 1/5 ---
Train size: 6493, Val size: 1616
Training fold 1 with early stopping (Device: gpu)...
[0]	validation_0-mlogloss:2.68243	validation_0-merror:0.64542
[100]	validation_0-mlogloss:1.21892	validation_0-merror:0.41584
[200]	validation_0-mlogloss:1.19025	validation_0-merror:0.40408
[300]	validation_0-mlogloss:1.18986	validation_0-merror:0.39790
[393]	validation_0-mlogloss:1.19732	validation_0-merror:0.39790
Fold 1 - Competition Score: 0.7481 (Binary F1: 0.9771, Macro F1: 0.5191)

--- Fold 2/5 ---
Train size: 6387, Val size: 1722
Training fold 2 with early stopping (Device: gpu)...
[0]	validation_0-mlogloss:2.68344	validation_0-merror:0.64866
[100]	validation_0-mlogloss:1.26048	validation_0-merror:0.43786
[200]	validation_0-mlogloss:1.24531	validation_0-merror:0.42102
[300]	validation_0-mlogloss:1.25626	validation_0-merror:0.41812
[400]	valida

In [17]:
# # Train models
# # models, cv_scores, overall_score = train_lightgbm_models(X_train, y_train, subjects)
# models, cv_scores, overall_score = train_xgboost_models(X_train, y_train, subjects)
# # Prepare feature columns for inference
# feature_cols = [col for col in X_train.columns if col != 'sequence_id']
    
   
# print(f"\n✓ Training completed successfully!")
# print(f"✓ Final CV Score: {overall_score:.4f}")
# print(f"✓ Models ready for inference")

In [18]:
joblib.dump(models, "xgb_model.pkl")
joblib.dump(feature_cols, "feature_cols.pkl")
joblib.dump(imu_cols, "imu_cols.pkl")
# joblib.dump(scaler, "scaler.pkl")

['imu_cols.pkl']

In [19]:
pl.Config.set_tbl_cols(-1)   # -1 means unlimited
pd.set_option("display.max_columns", None)
X_train

sequence_length  age  adult_child  sex  handedness  height_cm  \
0                  57   12            0    1           1      163.0   
1                  68   24            1    1           1      173.0   
2                  53   12            0    1           1      157.0   
3                  61   13            0    0           1      171.0   
4                  54   13            0    0           1      165.0   
...               ...  ...          ...  ...         ...        ...   
8104               76   52            1    1           1      170.0   
8105               84   31            1    0           1      184.0   
8106               78   12            0    1           1      157.0   
8107               48   36            1    0           1      175.0   
8108               53   30            1    0           1      186.0   

      shoulder_to_wrist_cm  elbow_to_wrist_cm  acc_x_mean  acc_x_std  \
0                       52               24.0    6.153098   1.322400   
1                       49               26.0    3.400506   1.079119   
2                       44               26.0   -7.058962   1.282907   
3                       54               26.0    5.524654   1.065268   
4                       52               23.0    5.363715   1.612496   
...                    ...                ...         ...        ...   
8104                    52               23.0   -1.247481   7.613847   
8105                    56               28.0    2.747396   3.296488   
8106                    44               26.0   -2.491286   6.881412   
8107                    54               25.0    7.054199   1.702883   
8108                    55               30.0    4.389298   1.148800   

      acc_x_var  acc_x_min  acc_x_max  acc_x_median  acc_x_q25  acc_x_q75  \
0      1.748742   3.613281   9.015625      6.488281   5.566406   6.988281   
1      1.164498   1.734375   5.906250      3.437500   2.536133   4.414062   
2      1.645850  -9.250000  -3.347656     -7.144531  -8.136719  -6.222656   
3      1.134795   3.437500   9.378906      5.390625   4.703125   6.234375   
4      2.600142   1.964844   6.832031      6.101562   5.756836   6.322266   
...         ...        ...        ...           ...        ...        ...   
8104  57.970672  -9.359375  12.167969     -6.732422  -7.340820   9.128906   
8105  10.866834  -1.433594   9.632812      1.476562  -0.525391   6.505859   
8106  47.353830 -10.023438   7.437500     -7.941406  -9.061523   4.634766   
8107   2.899811   2.414062   9.343750      7.796875   7.374023   8.011719   
8108   1.319742   2.546875   8.566406      4.078125   3.695312   5.000000   

      acc_x_iqr  acc_x_mean_abs  acc_x_rms  acc_x_range  acc_x_first  \
0      1.421875        6.153098   6.293596     5.402344     6.683594   
1      1.877930        3.400506   3.567623     4.171875     2.765625   
2      1.914062        7.058962   7.174594     5.902344    -5.972656   
3      1.531250        5.524654   5.626420     5.941406     6.621094   
4      0.565430        5.363715   5.600856     4.867188     4.300781   
...         ...             ...        ...          ...          ...   
8104  16.469727        7.449065   7.715367    21.527344     9.789062   
8105   7.031250        3.228888   4.291272    11.066406     6.535156   
8106  13.696289        6.963041   7.318493    17.460938     4.605469   
8107   0.637695        7.054199   7.256827     6.929688     8.273438   
8108   1.304688        4.389298   4.537145     6.019531     3.582031   

      acc_x_last  acc_x_delta  acc_x_skew  acc_x_kurt  acc_x_diff_mean  \
0       7.171875     0.488281   -0.545319   -0.391295         0.008719   
1       1.847656    -0.917969    0.146452   -1.069663        -0.013701   
2      -5.992188    -0.019531    0.518519    0.291585        -0.000376   
3       6.003906    -0.617188    0.747648    1.524114        -0.010286   
4       6.636719     2.335938   -1.397824    0.163483         0.044074   
...          ...          ...         ...         ... 

In [20]:
y_train

array([ 1,  4,  1, ...,  0,  1, 17])

In [21]:
subjects

array(['SUBJ_059520', 'SUBJ_020948', 'SUBJ_040282', ..., 'SUBJ_040282',
       'SUBJ_063447', 'SUBJ_039498'], dtype='<U11')

In [22]:
imu_cols

['subject',
 'rot_w',
 'row_id',
 'acc_z',
 'acc_x',
 'rot_z',
 'sequence_counter',
 'sequence_id',
 'rot_x',
 'acc_y',
 'rot_y']

In [23]:
feature_cols

['sequence_length',
 'age',
 'adult_child',
 'sex',
 'handedness',
 'height_cm',
 'shoulder_to_wrist_cm',
 'elbow_to_wrist_cm',
 'acc_x_mean',
 'acc_x_std',
 'acc_x_var',
 'acc_x_min',
 'acc_x_max',
 'acc_x_median',
 'acc_x_q25',
 'acc_x_q75',
 'acc_x_iqr',
 'acc_x_mean_abs',
 'acc_x_rms',
 'acc_x_range',
 'acc_x_first',
 'acc_x_last',
 'acc_x_delta',
 'acc_x_skew',
 'acc_x_kurt',
 'acc_x_diff_mean',
 'acc_x_diff_std',
 'acc_x_n_changes',
 'acc_x_time_corr',
 'acc_x_seg1_mean',
 'acc_x_seg2_mean',
 'acc_x_seg3_mean',
 'acc_x_seg1_std',
 'acc_x_seg2_std',
 'acc_x_seg3_std',
 'acc_x_seg1_to_seg2',
 'acc_x_seg2_to_seg3',
 'acc_x_peak_count',
 'acc_x_mad',
 'acc_x_range_norm',
 'acc_x_accel_mean',
 'acc_x_accel_std',
 'acc_x_cv',
 'acc_x_energy',
 'acc_y_mean',
 'acc_y_std',
 'acc_y_var',
 'acc_y_min',
 'acc_y_max',
 'acc_y_median',
 'acc_y_q25',
 'acc_y_q75',
 'acc_y_iqr',
 'acc_y_mean_abs',
 'acc_y_rms',
 'acc_y_range',
 'acc_y_first',
 'acc_y_last',
 'acc_y_delta',
 'acc_y_skew',
 'acc_